# Sprint22課題 自然言語処理入門

 ## この課題の目的

- 自然言語処理を体験する
- 簡単な文書の分析ができるようになる

In [1]:
import pandas as pd
import numpy as np
import re
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from nltk import word_tokenize, FreqDist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, GRU, Embedding, Input
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils, to_categorical
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from tensorflow.python.keras import models

Using TensorFlow backend.


## 【問題1】BoWとN-gram(手計算)

目的

- 古典的かつ強力な手法BoWとN-gramの理解

以下は俳優K.Kさんのつぶやき(コーパス)です。

【問】  
特殊文字除去(!や〜など)、単語分割をし以下の2パターンで文1〜文4を数値化(ベクトル化)してください。

- BoW(1-gram)  
- BoW(2-gram)  
手計算の後見やすい形にしてください。

例

In [2]:
vocabulary = ["I", "love", "this", "is", "the", "baseball"]
ms_kk_texts = ["I love baseball !!", "I love this !"]
texts_vec = [[1,1,0,0,0,1], [1,1,1,0,0,0]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram

,I,love,this,is,the,baseball
I love baseball !!,1,1,0,0,0,1
I love this !,1,1,1,0,0,0


### BoW(1-gram)

In [3]:
#特殊文字除去後の単語
vocabulary = ['今', '今日', 'ドラマ', '映画', '瞬', '撮影', '休憩', '中', '公開', 'です']

#文章
kk_texts = ['今撮影中で〜す！', 
                    '今休憩中で〜す(^^)', 
                    '今日ドラマ撮影で〜す！', 
                    '今日、映画瞬公開で〜す！！！']

texts_vec = [[1, 0, 0, 0, 0, 1, 0, 1, 0, 1], 
                        [1, 0, 0, 0, 0, 0, 1, 1, 0, 1],
                        [0, 1, 1, 0, 0, 1, 0, 0, 0, 1],
                        [0, 1, 0, 1, 1, 0, 0, 0, 1, 1]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = kk_texts)
df_bow_1gram

,今,今日,ドラマ,映画,瞬,撮影,休憩,中,公開,です
今撮影中で〜す！,1,0,0,0,0,1,0,1,0,1
今休憩中で〜す(^^),1,0,0,0,0,0,1,1,0,1
今日ドラマ撮影で〜す！,0,1,1,0,0,1,0,0,0,1
今日、映画瞬公開で〜す！！！,0,1,0,1,1,0,0,0,1,1


### BoW(2-gram)

In [4]:
#特殊文字除去後の単語
vocabulary = ['今撮影', '撮影中', '中です', '今休憩', '休憩中', '今日ドラマ', 'ドラマ撮影', 
                          '撮影です', '今日映画', '映画瞬', '瞬公開', '公開です']

#文章
kk_texts = ['今撮影中で〜す！', 
                    '今休憩中で〜す(^^)', 
                    '今日ドラマ撮影で〜す！', 
                    '今日、映画瞬公開で〜す！！！']

texts_vec = [[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                        [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = kk_texts)
df_bow_1gram

,今撮影,撮影中,中です,今休憩,休憩中,今日ドラマ,ドラマ撮影,撮影です,今日映画,映画瞬,瞬公開,公開です
今撮影中で〜す！,1,1,1,0,0,0,0,0,0,0,0,0
今休憩中で〜す(^^),0,0,1,1,1,0,0,0,0,0,0,0
今日ドラマ撮影で〜す！,0,0,0,0,0,1,1,1,0,0,0,0
今日、映画瞬公開で〜す！！！,0,0,0,0,0,0,0,0,1,1,1,1


## 【問題2】TF-IDF(手計算)

目的

- 古典的かつ強力なTF-IDFの理解
標準的なTF-IDFの公式 

Term Frequency:

$$tf(t,d) = \frac{n_{t,d}}{\sum_{s \in d}n_{s,d}}$$

$n_{t,d}$ : 文書d内の単語tの出現回数

$\sum_{s \in d}n_{s,d}$ : 文書dの全単語の出現回数の和

Inverse Document Frequency:

$$idf(t) = \log_2{\frac{N}{df(t)}}$$

$N$ : 全文書数

$df(t)$ : 単語tが出現する文書数

TF-IDF:

$$tfidf(t, d) = tf(t, d) \times idf(t)$$

【問】
問題1のコーパスを使って、文1〜文4をTFIDFで数値化(ベクトル化)してください。   
問題1と同様、手計算の後見やすい形にしてください。

正解例  
tfidf(今, 文書1) = 0.25 になります。

In [5]:
def tf_idf(t_num, t_sum, n, df):
    '''
    TF-IDFの計算
    
    Parameters
    t_num : int
        文書d内の単語tの出現回数
    t_sum : int
        文書dの全単語の出現回数の和
    n : int
         全文書数
    df : int
        単語tが出現する文書数
    '''
    tf = t_num / t_sum
    
    idf = np.log2(n / df)
    
    return tf * idf

In [6]:
#特殊文字除去後の単語
vocabulary = ['今', '今日', 'ドラマ', '映画', '瞬', '撮影', '休憩', '中', '公開', 'です']

#文章
kk_texts = ['今撮影中で〜す！', 
                    '今休憩中で〜す(^^)', 
                    '今日ドラマ撮影で〜す！', 
                    '今日、映画瞬公開で〜す！！！']

In [7]:
#tf_idfを求める(単語_文章番号)
#文章1
ima_1 = tf_idf(1, 4, 4, 2)
satuei_1 = tf_idf(1, 4, 4, 2)
tyuu_1 = tf_idf(1, 4, 4, 2)
desu_1 = tf_idf(1, 4, 4, 4)

#文章2
ima_2 = tf_idf(1, 4, 4, 2)
kyuukei_2 = tf_idf(1, 4, 4, 1)
tyuu_2 = tf_idf(1, 4, 4, 2)
desu_2 = tf_idf(1, 4, 4, 4)

#文章3
kyou_3 = tf_idf(1, 4, 4, 2)
drama_3 = tf_idf(1, 4, 4, 2)
satuei_3 = tf_idf(1 ,4, 4, 2)
desu_3 = tf_idf(1, 4, 4, 4)

#文章4
kyou_4 = tf_idf(1, 5, 4, 2)
eiga_4 = tf_idf(1, 5, 4, 1)
matataki_4 = tf_idf(1, 5, 4, 1)
koukai_4 = tf_idf(1, 5, 4, 1)
desu_4 = tf_idf(1, 5, 4, 4)

In [8]:
texts_vec = [[ima_1, 0, 0, 0, 0, satuei_1, 0, tyuu_1, 0, desu_1], 
                        [ima_2, 0, 0, 0, 0, 0, kyuukei_2, tyuu_2, 0, desu_2],
                        [0, kyou_3, drama_3, 0, 0, satuei_3, 0, 0, 0, desu_3],
                        [0, kyou_4, 0, eiga_4, matataki_4, 0, 0, 0, koukai_4, desu_4]]

df_bow_tfidf = pd.DataFrame(data = texts_vec, columns = vocabulary, index = kk_texts)
df_bow_tfidf

,今,今日,ドラマ,映画,瞬,撮影,休憩,中,公開,です
今撮影中で〜す！,0.25,0.00,0.00,0.0,0.0,0.25,0.0,0.25,0.0,0.0
今休憩中で〜す(^^),0.25,0.00,0.00,0.0,0.0,0.00,0.5,0.25,0.0,0.0
今日ドラマ撮影で〜す！,0.00,0.25,0.25,0.0,0.0,0.25,0.0,0.00,0.0,0.0
今日、映画瞬公開で〜す！！！,0.00,0.20,0.00,0.4,0.4,0.00,0.0,0.00,0.4,0.0


## 【問題3】テキストクリーニング(プログラミング)
目的

- 実データ対応のためのテキストクリーニングの理解
- 正規表現の理解

実際のテキストデータは非常に汚いことが多いです。   
以下は3/6(水)にnoroさんがSlackで発言した文章で、良い例です。

```
<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:
```

【問】
このテキストに以下の処理を施してください。

- urlを削除
- 【〇〇】を削除
- 改行等の特殊文字を削除
- 絵文字除去

ここではしませんが、数字を文字列NUMBERに置き換える処理をよくします。

正解例

```
有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！
```

正規表現のサンプル

In [9]:
# 対象テキストデータ
text = '【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\n'
# re.compileを使うと処理が早くなります
BAD_SYMBOL = re.compile('[\n*！`]+')
# re.sub(r'[\n*！`]+', '', text)でもできます
text = re.sub(BAD_SYMBOL, '', text)
text

'【スペシャル特典】有償のRubyMineやPyCharmの 6ヶ月間100%OFFクーポン をご希望者の方先着100名様に贈呈いたします'

In [10]:
text = '<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFF'\
            'クーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineや'\
            'PyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポン'\
        'をいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>'\
        '\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、'\
        '手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、'\
        '*`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:'

In [11]:
#<>
url = '<.*?>'

#【】
kakko = '【.*?】'

#特殊文字
tokusyu = '[\n\*`\s]'

#絵文字
emoji = ':[^0-90-9]+:'

#連結
reg_str = f'{url}|{kakko}|{tokusyu}|{emoji}'

In [12]:
#削除
remove = re.compile(reg_str)
text = re.sub(remove, '', text)
print(text)

有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！


リアルタイムで正規表現を確認できるサイトです。

[https://regex101.com/](https://regex101.com/)

[https://regex-testdrive.com/ja/dotest](https://regex-testdrive.com/ja/)

[re 正規表現操作](https://docs.python.org/ja/3/library/re.html)

## Tips NLPのLinuxコマンド

これまでpythonでファイルを読み込んで処理をしていましたが、  
簡単な作業においてはlinuxコマンドの方がメモリの使用料が半分以下だったりとパフォーマンスが良いです。

例えばファイルの行数を数えたい場合、pythonでわざわざ書くのは面倒です。  
以下の1行のコマンドで実行できます。

```
wc -l 〇〇.txt
```

また分割したい場合はsplit  
並び替えたい場合はsort  
置換にはsed  
文の先頭、後頭部分を見たければhead,tail  
など便利なコマンドがあります。  
詳しく知りたい方はNLP100本ノックで調べてみてください。

## 【問題4】形態素解析

目的

- 形態素解析の理解

形態素解析のツールはMecabやJanomeなど様々ですが、  
ここでは手軽に導入できるJanomeを使います。

[Janome document](https://mocobeta.github.io/janome/)

【問】

上記のクリーニングしたテキストをJanomeを用いて形態素解析をし、  
名詞または動詞の単語を抜き出してください。

正解例

```
["有償", "RubyMine", "Pycharm", ...]
```

In [13]:
#動詞or名詞なら抜き出す
t = Tokenizer()
print([token.surface for token in t.tokenize(text) if token.part_of_speech.split(',')[0] in ['動詞', '名詞']])

['有償', 'RubyMine', 'PyCharm', '6', 'ヶ月', '間', '100', '%', 'OFF', 'クーポン', '希望', '者', '方', '先着', '100', '名', '様', '贈呈', 'いたし', '度', 'RubyMine', 'PyCharm', 'メーカー', 'JetBrains', '社', 'クーポン', 'コード', '提供', '交渉', '実り', '100', 'クーポン', 'いただく', 'こと', 'でき', 'RubyMinePyCharm', '希望', '方', '手', '挙げ', '方式', '希望', '方', '投稿', '手', 'あげ', 'スタンプ', 'クリック', 'し', 'ください', '期限', '2019', '年', '3', '月', '20', '日', '水', '22', ':', '00', 'さ', 'せ', 'いただき', '希望', 'お待ち', 'し', 'おり']


**複合名詞化の場合**

In [14]:
#動詞or名詞なら抜き出す
a = Analyzer(token_filters=[CompoundNounFilter()])
print([token.surface for token in a.analyze(text) if token.part_of_speech.split(',')[0] in ['動詞', '名詞']])

['有償', 'RubyMine', 'PyCharm', '6ヶ月間100%OFFクーポン', '希望者', '方先着100名様', '贈呈', 'いたし', '度', 'RubyMine', 'PyCharm', 'メーカー', 'JetBrains社', 'クーポンコード', '提供交渉', '実り', '100クーポン', 'いただく', 'こと', 'でき', 'RubyMinePyCharm', '希望', '方', '手', '挙げ', '方式', '希望', '方', '投稿', '手', 'あげ', 'スタンプ', 'クリック', 'し', 'ください', '期限', '2019年3月20日', '水', '22:00', 'さ', 'せ', 'いただき', '希望', 'お待ち', 'し', 'おり']


## 【問題5】ニュースの分析

目的

- 日本語の自然言語処理の体験
- 類似度の理解

以下からldcc-20140209.tar.gzをダウンロードしてください。   
[livedoor](https://www.rondhuit.com/download.html#ldcc)

もしくはwgetコマンドを使っても良いです。

```
# livedoorのnewsをダウンロード
wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# 圧縮ファイルを解凍
tar zxf ldcc-20140209.tar.gz
# livedoorニュースの説明を表示
cat text/README.txt
```

In [15]:
# encodingをutf-8指定して読み込み
bin_data = load_files('./text', encoding='utf-8')
documents = bin_data.data
# 今回はラベルが無いと仮定してください
# targets = bin_data.target

【問】

以下の流れでニュースを分析してください。

- まずどんなニュースなのか読んでみる
- 出現単語をカウントして分析する
- テキストをクリーニングする
- BoW + TFIDFでベクトル化する
- あるニュースに一番cos類似度が近いニュースを出力する関数の作成
- 別の類似度手法を1つ調べて上の関数に組み込む(切り替えられるようにする)
- なぜそのような結果になったのか考察する

[sklearn.feature_extraction.text](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [16]:
#最初のニュースの内容
documents[0]

'http://news.livedoor.com/article/detail/4931238/\n2010-08-08T10:00:00+0900\nNY名物イベントが日本でも！名店グルメを気軽に楽しむ\nニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。\n\u3000\n\u3000実行委員長には、学校法人服部学園、服部栄養専門学校 理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LA BETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。\n\n参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストラン タテル ヨシノ 汐留」や、日本料理の名門「つきぢ田村」、「金田中 庵」、「赤坂璃宮」に「mikuni MARUNOUCHI」など、日本を代表するレストランがずらり。\n\u3000イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。\n\n\u3000予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？\n\nJAPAN RESTAURANT WEEK 2010 -公式サイト\n'

In [17]:
#最初の文章の出現単語
a = Analyzer(token_filters=[TokenCountFilter()])

g_count = list(a.analyze(documents[0]))

print(g_count)

[('http', 1), ('://', 1), ('news', 1), ('.', 2), ('livedoor', 1), ('com', 1), ('/', 4), ('article', 1), ('detail', 1), ('4931238', 1), ('\n', 6), ('2010', 5), ('-', 3), ('08', 2), ('T', 1), ('10', 1), (':', 2), ('00', 2), ('+', 1), ('0900', 1), ('NY', 1), ('名物', 1), ('イベント', 4), ('が', 7), ('日本', 4), ('で', 6), ('も', 2), ('！', 3), ('名店', 1), ('グルメ', 1), ('を', 11), ('気軽', 2), ('に', 14), ('楽しむ', 1), ('ニューヨーク', 1), ('20', 1), ('年', 1), ('続い', 1), ('て', 7), ('いる', 2), ('食', 2), ('の', 17), ('祭典', 1), ('「', 8), ('レストラン', 7), ('・', 8), ('ウィーク', 3), ('」', 8), ('。', 7), ('その', 1), ('版', 3), ('ダイナーズクラブ', 1), ('特別', 3), ('協賛', 2), ('もと', 1), ('7', 1), ('月', 3), ('30', 1), ('日', 3), ('より', 1), ('スタート', 1), ('8', 2), ('31', 1), ('まで', 2), ('期間', 3), ('中', 1), ('、', 28), ('青山', 1), ('六本木', 1), ('丸の内', 1), ('銀座', 1), ('横浜', 1), ('エリア', 1), ('から', 1), ('ラグジュアリーレストラン', 1), ('81', 1), ('店舗', 2), ('この', 1), ('ため', 1), ('用意', 1), ('し', 3), ('た', 2), ('ランチ', 1), ('メニュー', 1), ('円', 2), ('（', 2), ('税', 2), ('サ

In [18]:
#<>
kakko1 = '<.*?>'

#【】
kakko2 = '【.*?】'

#特殊文字
tokusyu = '[\n\*`\s■◆○★●]'

#URL
url = '(http.*?\n)'

#日時
date = '(\d{4}-.*?\+\d{4})'

#連結
reg_str = f'{kakko1}|{kakko2}|{tokusyu}|{url}|{date}'

In [19]:
#テキストクリーニングしてリストに戻す
document = []

for i in documents:
    remove = re.compile(reg_str)
    document.append(re.sub(remove, '', i))

#最初のニュース
print(document[0])

NY名物イベントが日本でも！名店グルメを気軽に楽しむニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。実行委員長には、学校法人服部学園、服部栄養専門学校理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LABETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストランタテルヨシノ汐留」や、日本料理の名門「つきぢ田村」、「金田中庵」、「赤坂璃宮」に「mikuniMARUNOUCHI」など、日本を代表するレストランがずらり。イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？JAPANRESTAURANTWEEK2010-公式サイト


In [20]:
#形態素解析してリストに戻す(複合名詞　+ 動詞)
corpus = []

a = Analyzer(token_filters=[CompoundNounFilter()])
for text in tqdm_notebook(document):
    corpus.append(' '.join([token.surface for token in a.analyze(text) 
                                    if token.part_of_speech.split(',')[0] in ['動詞', '名詞']]))

In [21]:
#形態素解析後のニュース
print(corpus[0])

NY名物イベント 日本 名店グルメ 気軽 楽しむ ニューヨーク 20年 続い いる 食 祭典 レストラン ウィーク 日本版 ダイナーズクラブ特別協賛 もと7月30日 スタート 8月31日 期間中 青山 六本木 丸の内 銀座 横浜 エリア ラグジュアリーレストラン81店舗 イベント ため 特別用意 し ランチメニュー2010円 税 サ別 ディナー5000円 税 サ別 気軽 楽しめる とっ おき イベント 実行委員長 学校法人服部学園 服部栄養専門学校理事長 校長 医学博士 服部幸應氏 迎え 実行委員 石田純一さん LABETTOLAオーナーシェフ落合務氏 フードアナリスト協会会長 高賀右近氏 つき ぢ田村三代目 田村隆氏 放送作家 脚本家 小山薫堂さん 食 スペシャリストたち 勢揃い 参加レストラン ミシュラン フランス版 東京版とも 星 獲得 し いる 吉野建シェフ レストランタテルヨシノ汐留 日本料理 名門 つき ぢ田村 金田中庵 赤坂璃宮 mikuniMARUNOUCHI 日本 代表 する レストラン イベント期間 8月19日 特別協賛 ダイナーズクラブカード会員 シティバンク 口座 持つ シティゴールドメンバー 楽しめる 先行期間 なり その後 誰 参加 できる 日程 チェック 必須 予約方法 事前 店舗 問合せ 行い ジャパンレストラン ウィーク2010 予約 こと 伝えれ OK 憧れ い レストラン 料理 リーズナブル いただける チャンス 極上 味 ラグジュアリー 空間 満喫 幸せ 実感 できる ジャパンレストラン ウィーク2010 参加 し て み いかが JAPANRESTAURANTWEEK2010-公式サイト


In [22]:
#BoW
count = CountVectorizer()

docs = np.array(corpus)
bag = count.fit_transform(docs)
print(bag.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [23]:
#TF-IDF
tfidf = TfidfTransformer()
vector = tfidf.fit_transform(bag).toarray()
print(vector)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [202]:
def similar(documents, news_num=1, sim='cos'):
    '''
    cos類似度を求める
    
    Parameters
    --------------
    documents : リスト
        様々なニュース
    news_num : int
        何番目のニュースか(デフォルト : 1(1番目のニュース))
    sim : str
        類似度(デフォルト : cos(cos類似度))
    ''' 
    #様々なニュース
    #形態素解析してリストに戻す(複合名詞　+ 動詞)
    corpus = []
    a = Analyzer(token_filters=[CompoundNounFilter()])
    for document in tqdm_notebook(documents):
        corpus.append(' '.join([token.surface for token in a.analyze(document) 
                                        if token.part_of_speech.split(',')[0] in ['動詞', '名詞']]))
    
    #Bow
    count = CountVectorizer()
    docs = np.array(corpus)
    bag = count.fit_transform(docs)
    
    #TF-IDF
    fidf = TfidfTransformer()
    vector = tfidf.fit_transform(bag).toarray()
    
    #ターゲットのニュース
    news_num -= 1 #ニュースの番号
    news = docs[news_num] #ニュースの内容
    news_corpus = corpus[news_num] #形態素解析したニュース
    news_vector = vector[news_num] #TF-IDFしたニュース
    

    #cos類似度の場合
    if sim == 'cos':
        
        #初期値
        cos_max = 0 #最大のcos類似度
        index = 0 #cos類似度が最大の時のindex

        #cos類似度を計算
        for i, doc_vec in enumerate(vector):        
            cos = np.dot(news_vector, doc_vec) / (np.linalg.norm(news_vector) * np.linalg.norm(doc_vec))

            #同じ文章ならpass
            if int(cos) == 1:
                pass

            #cos類似度が最高ならcos類似度とインデックスを更新
            else:
                if cos_max < cos:
                    cos_max = cos
                    index = i

        print('cos類似度　:　{:.3}'.format(cos_max) + '\n' + documents[index])
        
        
    #doc2vec
    elif sim == 'doc2vec':
        
        #初期値
        training_docs = [] 

        #ターゲットのニュースをリストに入れる
        sent1 = TaggedDocument(words=news_corpus.split(' '), tags=[news])
        training_docs.append(sent1)
        
        #全てのニュースをリストに入れる
        for i, (doc_corpus, doc) in enumerate(zip(corpus, documents)):
            
            #ターゲットのニュースと同じニュースはpass、それ以外はリストへ
            if i == news_num:
                pass
            
            else:
                sent2 = TaggedDocument(words=doc_corpus.split(' '), tags=[doc])
                training_docs.append(sent2)

        #類似度を求め、最も高いものを出力
        model = Doc2Vec(documents=training_docs)
        similar = model.docvecs.most_similar(news, topn=1)
        
        print(similar)

**cos類似度**

In [203]:
#1番最初のニュースと似ているニュースを探す
similar(document, news_num=1, sim='cos')


cos類似度　:　0.0727
一度参加すればハマる？噂の“街コン”に参加してみた！昨年あたりから日本各地で流行している街ぐるみの合コンイベント“街コン”。有名なのは栃木の宇都宮や広島、福島といったところで、男女合わせて2000人以上が参加しているらしい。なぜこんなに多くの人が参加するのか？興味はあったが、筆者は首都圏在住なので参加するにはちょっと遠いのだ。しかしここ最近は東京をはじめとする首都圏でも、数多くの街コンが開催されているという。1000人規模とはいかなくても、200人とか400人とか、どこもそれなりに規模は大きい様子。これはぜひ参加して婚活せねば！ということで早速某駅周辺のイベントに参加してきたのだが……いやーすごかった。まず参加してビックリしたのは、皆のやる気。受付開始からイベントがスタートするまで約2時間あったのだが、余裕を持って1時間前に到着したらもう大行列！聞くと受付開始30分過ぎにはもうこんな状態だったという。あらら〜と後悔しても仕方がないので大人しく並び受付を済ませると、リストバンドと地図が配布される。リストバンドは参加者の目印で、これがあれば街でイベントに参加しているいくつかの店舗に自由に出入りすることができ、飲み食いし放題！そしてその店の情報が掲載されているのが地図。これをみながら探索するという仕組みだ。さっそく店に入って席につくと、案内の人が参加者の男性と同席できるようセッティングしてくれた。そこでまずご飯を食べながら交流がスタート。「どこから来ました？」「年齢は？」「社会人ですか？」といった自己紹介がされ、あとはお互いを探り探りトークへ突入する。ここまでは普通の合コンと大差ないが、大きく違うのは“気に入らなかったら他にいける”ということ。そういう場合は「他の店にいってみたいので〜」などと適当に理由をつけて店を出ちゃえばいいのだ。で、他の店に行けばまた違う男性と同席が可能。つまり、店ごとに違う合コンを何度もやっている感覚である。これは確かに新しい。さらに男性との交流以外にも料理を食べたり飲んだりできるという楽しさもあるし、男性以外にも女性の友達を作るのもアリだ。実際筆者も一番楽しかったのは、バーカウンターで一緒になった同年代の女性たちとのトーク。皆男性をそっちのけで「いやー、いい出会いってないよねー」といった婚活話に花を咲かせる始末で

**doc2vec**

In [205]:
#1番最初のニュースと似ているニュースを探す(文章、類似度の順で出力)
similar(document, news_num=1, sim='doc2vec')


[('「デルレイ」のコラボショコラデザートコースで、今年は特別なバレンタインクリスマス、忘年会、新年会がひと段落したと思ったら、気付けばバレンタインデーまであと1ヶ月あまり。大切なあの人への贈り物はとびっきり特別にしたいのに、なんとなく毎年同じようなショコラを選んでしまう…。そんな想いをした経験がある女子も多いのでは無いでしょうか？そんな方に朗報です！“しあわせのリンク＆スマイル”をテーマにいつも最上級のくつろぎを提案するホテル日航東京が、高級老舗ショコラティエ「デルレイ」とコラボレーションした世界初の「ショコラデザートコース」を提供。コース仕立てで楽しむチョコレートの味わいが、2人の時間をさらに幸せに演出します。デザートをコースで楽しむなんて、とても贅沢！気になる中身は、濃厚なカカオが香るあたたかいチョコレートドリンク「ショコラ・ショー」、ほどよい酸味が心地よい季節のフルーツ「あまおう苺」を使用した、ホテルオリジナルの「あまおう苺とシャンパンジュレ」とバラエティに富んだスイーツが次々に登場。3品目には、「デルレイ」の得意技である、「フルーツフランベ」。シャンパンとフレッシュフルーツを加え仕上げをしてくれるので、香りにうっとりと酔いしれることができます。そして、「デルレイ」のアイコンとも言える燦然と輝くダイヤモンド、デルレイ監修によるホテルオリジナルのショコラマカロン、ガトーショコラの「プレミアムショコラプレート」で大満足のラスト。このコースが味わえるラウンジ＆シャンパンバー「ベランダ」は、レインボーブリッジが見えるキラッキラの夜景が魅力的。常時置いてある、14種類のシャンパンの他、「ショコラデザートコース」にぴったりな特別銘柄が期間限定で登場。シャンパンと味わう新しいショコラの楽しみ方で、今年のバレンタインデーはちょっぴり差をつけてみませんか？「ショコラデザートコース」期間：2010年2月1日（月）〜2月14日（日）レストラン：ラウンジ＆シャンパンバー「ベランダ」（ティータイム10：00〜17：00／バータイム17：00〜23：00）コース内容：ショコラデザートコース3,150円※消費税込み、サービス料別・あまおう苺とシャンパンジュレシャンパンケーキを添えて・デルレイショコラ・ショーとプティ・フール・セック・シャンパンフルーツシュゼット・プレミアムショコラプレー

1番最初のニュースは、レストランのイベント開催に関するニュースであり、これと似ているニュースを検索した。
cos類似度の場合、街コンに関するニュースが出力され、このときの類似度は0.0727であった。これはイベントという部分は同じだが、cos類似度が0.0727と低いため、あまり似ているとは言えない。

doc2vecの場合、ホテルのレストランでのバレンタインイベントに関するニュースが出力され、この時の類似度0.9478はであった。こちらはどちらもレストランのイベント開催のお知らせという共通点があり、類似度も0.9478と非常に高かった。

## 【問題6】感情分析

目的

- NLP定番の感情分析の経験
- 英語の処理の実践

以下からLarge Movie Review Datasetをダウンロードしてください。

[IMDBレビュー](http://ai.stanford.edu/~amaas/data/sentiment/)

同じようにwgetコマンドでも可能です。

In [24]:
#データの読み込み
train_review = load_files('./aclImdb/train/', encoding='utf-8')
train_x, train_y = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
test_x, test_y = test_review.data, test_review.target

【問】

IMDBという映画に対するレビューのデータセットを使います。  
良いレビューか悪いレビューかを判定するモデルを作ってください。  
テストデータに対する正解率が90%を超えるまで、調査=>実行=>改善を繰り返してください。  
前処理になぜその処理をしたのかを書くとエンジニアリングとしても完璧です。

注意: 必ず間違っていたデータを観察してください。

In [25]:
#1番最初の学習用データ
print(train_x[0])
print(train_y[0])

Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.
1


In [26]:
#学習用とテスト用のデータをくっつける
train_x[len(train_x): len(train_x)] = test_x
len(train_x)

50000

In [27]:
#<br>
br = '<.*?>'

#特殊文字
tokusyu = "[\?!\/,.'\(\):\"-;]"

#連結
reg_str = f'{br}|{tokusyu}'

In [28]:
#テキストクリーニングしてリストに戻す
text = []

for i in train_x:
    #ぜんぶ小文字に
    i = i.lower()
    
    #正規表現で削除
    remove = re.compile(reg_str)
    text.append(re.sub(remove, ' ', i))

#最初のニュース
print(text[0])

zero day leads you to think  even re think why two boys young men would do what they did   commit mutual suicide via slaughtering their classmates  it captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own mutual world via coupled destruction   it is not a perfect movie but given what money time the filmmaker and actors had   it is a remarkable product  in terms of explaining the motives and actions of the two young suicide murderers it is better than  elephant    in terms of being a film that gets under our  rationalistic  skin it is a far  far better film than almost anything you are likely to see    flawed but honest with a terrible honesty 


In [29]:
#学習用とテスト用に戻す
X_train = text[:25000]
X_test = text[25000:]

In [30]:
NGRAM_RANGE = (1, 2)
TOP_K = 10000
TOKEN_MODE = 'word'
MIN_DOC_FREQ = 2

def ngram_vectorize(train_texts, train_labels, test_texts):
    kwargs = {
        'ngram_range' : NGRAM_RANGE,
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : TOKEN_MODE,
        'min_df' : MIN_DOC_FREQ,
    }
    
    # Learn Vocab from train texts and vectorize train and val sets
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    x_train = tfidf_vectorizer.fit_transform(train_texts)
    x_test = tfidf_vectorizer.transform(test_texts)
    
    # Select best k features, with feature importance measured by f_classif
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_test = selector.transform(x_test).astype('float32')
    return x_train, x_test

In [31]:
X_train, X_test = ngram_vectorize(X_train, train_y, X_test)

/Users/yuhisoejima/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1577: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [32]:
# 入力の形式は映画レビューで使われている語彙数
vocab_size = X_train.shape[1:]

#モデル
model = Sequential()
model.add(Dropout(0.5, input_shape=vocab_size, ))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 10000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                160016    
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [33]:
#コンパイル
model.compile(optimizer=Adam(lr=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

#コールバック
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]

In [34]:
#学習
history = model.fit(X_train,
                    train_y,
                    epochs=10,
                    batch_size=128,
                    validation_data=(X_test, test_y), 
                    verbose=1,
                    callbacks=callbacks)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 3s 115us/step - loss: 0.6410 - acc: 0.6543 - val_loss: 0.4963 - val_acc: 0.8695
Epoch 2/10
25000/25000 [==============================] - 3s 109us/step - loss: 0.4296 - acc: 0.8413 - val_loss: 0.3070 - val_acc: 0.8958
Epoch 3/10
25000/25000 [==============================] - 3s 106us/step - loss: 0.3326 - acc: 0.8790 - val_loss: 0.2660 - val_acc: 0.9017
Epoch 4/10
25000/25000 [==============================] - 3s 102us/step - loss: 0.3051 - acc: 0.8897 - val_loss: 0.2513 - val_acc: 0.9049
Epoch 5/10
25000/25000 [==============================] - 3s 101us/step - loss: 0.2862 - acc: 0.8975 - val_loss: 0.2474 - val_acc: 0.9041
Epoch 6/10
25000/25000 [==============================] - 3s 100us/step - loss: 0.2718 - acc: 0.9017 - val_loss: 0.2471 - val_acc: 0.9019
Epoch 7/10
25000/25000 [==============================] - 3s 100us/step - loss: 0.2615 - acc: 0.9063 - val_loss: 0.2447 - 

In [37]:
#結果
history = history.history
test_acc = history['val_acc'][-1]
test_loss = history['val_loss'][-1]

print('loss :', test_loss)
print('acc :', test_acc)

loss : 0.24373817365646364
acc : 0.903919999961853


In [65]:
#間違ったデータの確認
np.where((np.where(model.predict(X_test) < 0.5, 0, 1).reshape(-1) == test_y) == False)[0][:5]

array([ 0, 11, 32, 37, 43])

いい評価を悪い評価と推定

In [67]:
print(test_x[0])
print(test_y[0])

Don't hate Heather Graham because she's beautiful, hate her because she's fun to watch in this movie. Like the hip clothing and funky surroundings, the actors in this flick work well together. Casey Affleck is hysterical and Heather Graham literally lights up the screen. The minor characters - Goran Visnjic {sigh} and Patricia Velazquez are as TALENTED as they are gorgeous. Congratulations Miramax & Director Lisa Krueger!
1


hateやfunなど、negativeな言葉とpositiveな言葉が入り混じっていた。

悪い評価をいい評価と推定

In [72]:
print(test_x[32])
print(test_y[32])

I would not recommend this movie. Even though it is rated G and is clearly for kids there is quite a lot of swearing (including the dreaded 'F' and 'S' words). This kind of language doesn't offend me particularly but in a kids film? Come on.<br /><br />There was also quite a bit of implied sexual content, between one of the early adolescent male characters and any willing adult woman who came along - including a prostitute! <br /><br />The acting was as good as it gets in this genre of film but the story line was very very cheesy and even my four year old remarked that it was 'stupid'.<br /><br />Despite having Elizabeth Shue, this film is definitely not worth checking out if you haven't seen it.
0


これもgoodやstupidなど、negativeな言葉とpositiveな言葉が入り混じっていたが、形容詞をあまり使わず批判を行なっている文章であると感じた。

## 【問題7】分散表現(アドバンス課題)

目的

- 主流である分散表現の理解

[分散表現の詳細はこちらを参考にしてください]()

【問】
以下の中から一つ選んで分散表現を獲得し、  
好きな単語群をt-SNE,PCAなどを用いて可視化してください。  
コーパスは自由です。

- Word2Vec-CBoW(2〜3人)
- Word2Vec-skip-gram(2〜3人)
- fastText(2〜3人)

また以下の4点についてもノートに書いてください。

- 分布仮説とは何か？
- 分散表現を得ることのメリットは何か？
- 上で選んだモデルのメリット、デメリットは何か？
- なぜそのパラメータを選んだのか？

## 【問題8】自然言語処理の応用事例

目的

- NLPの情報共有
現在自然言語処理はどのような企業でどのように活用されているか？   
1つ例をあげて3~5分で発表してください。   
(例)メルカリは商品説明をTF-IDFを用いてベクトル化して商品の異常検知を行っている。

Gunosyでは、自然言語処理を用いて記事を政治やスポーツ等に分類しており、各ユーザーに適している記事を配信している。

[Gunosy における AWS 上での自然言語処理・機械学習の活用事例](https://www.slideshare.net/keisukeosone/gunosy-aws-76649598)

## 深く学びたい方へ

- [首都大学小町研/推薦書籍](http://cl.sd.tmu.ac.jp/prospective/readings)
- [NAIST勉強会](http://www.phontron.com/teaching.php?lang=ja)
- [NLP100本ノック](http://www.cl.ecei.tohoku.ac.jp/nlp100/)
- [ACL](https://acl2018.org/)
- [NMNLP](https://emnlp2018.org/)
- [おすすめNLPデータセット](https://gengo.ai/ja/datasets/the-best-25-datasets-for-natural-language-processing/)